In [ ]:
# !pip install sklearn_pandas

In [ ]:
from sys import modules
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import sys
from pathlib import Path
import pandas as pd
import wandb
import joblib
from source.mlops.modules.download import *
from source.mlops.modules.preprocessing import *
from source.mlops.modules.train import *


In [ ]:
data = {
                "BMI": 16.4,
                "Smoking": "Yes",
                "AlcoholDrinking": "No",
                "Stroke": "No",
                "PhysicalHealth": 3.0,
                "MentalHealth": 30.0,
                "DiffWalking": "No",
                "Sex": "Female",
                "AgeCategory": "55-59",
                "Race": "White",
                "Diabetic": "Yes",
                "PhysicalActivity": "Yes",
                "GenHealth": "Very good",
                "SleepTime": 5.0,
                "Asthma": "Yes",
                "KidneyDisease": "No",
                "SkinCancer": "Yes",
            }



run = wandb.init(project="project_heart", job_type="api")
artifact = run.use_artifact(
    "diego25rm/project_heart/model_export:v3", type="pipeline_artifact"
).file()
loaded_model = joblib.load(artifact)
recieve = dict(data)
X = pd.DataFrame([data])
print(X)
prediction = loaded_model.predict(X)

In [ ]:
prediction

## Tunning the decision tree

In [ ]:
file_dir = Path().resolve().parent
file_dir = os.path.join(file_dir, 'data')
file_dir = os.path.join(file_dir, 'heart_2020_cleaned.csv')
data = pd.read_csv(file_dir)
y=LabelEncoder().fit_transform(data['HeartDisease'].values.reshape(-1,1))
y = pd.DataFrame(y,columns=['HeartDisease'])

X=data.drop(columns=['HeartDisease'])
pipe = pipeline_preprocessing()
pipe.fit_transform(X,y)

In [ ]:
X

In [ ]:
file_dir = Path().resolve().parent
file_dir = os.path.join(file_dir, 'data')
file_dir = os.path.join(file_dir, 'heart_2020_cleaned.csv')
data = pd.read_csv(file_dir)

model_artifact = "model_export"
model_pipeline = Pipeline(steps=[("preprocess",pipeline_preprocessing()),
                            ("train",DecisionTreeClassifier(criterion= 'gini', splitter= 'best',max_depth= 4))])
X=data.drop(columns=['HeartDisease'])
y=LabelEncoder().fit_transform(data['HeartDisease']).astype(float)
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)
model_pipeline.fit(X_train,y_train)

In [ ]:
X_test

In [ ]:
pd.DataFrame(model_pipeline.predict(X_test))[0].value_counts()

In [ ]:
# parameters = {'penalty':['l1', 'l2', 'elasticnet', 'none'], 'solver':['newton-cg','lbfgs', 'liblinear', 'sag', 'saga']}
# clf = GridSearchCV(LogisticRegression(), parameters,cv=3,scoring='precision')
# target = treat_data.HeartDisease
# features = treat_data.drop(columns=['HeartDisease'])
# clf.fit(features, target)
# print(clf.best_params_)

In [ ]:
from preprocessing import *
y = data['HeartDisease']
X = data.drop(columns=['HeartDisease'])
pipe = pipeline_preprocessing()

In [ ]:

X_prep = pipe.fit_transform(X=data, y=data['HeartDisease'])


In [ ]:
X_prep

In [ ]:
parameters = {'criterion':['gini','entropy'], 'splitter':['best', 'random'],'max_depth':range(1,8)}
clf = GridSearchCV(DecisionTreeClassifier(), parameters,cv=5,scoring='accuracy',error_score='raise')
y_prep = X_prep['HeartDisease']
X_prep = X_prep.drop(columns=['HeartDisease'])
clf.fit(X_prep, y_prep)
print(clf.best_params_)

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(criterion= 'gini', splitter= 'best',max_depth= 7)

In [ ]:
y = y_prep
X = X_prep
features_names = X.columns
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.express as px
dt = dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
cm = confusion_matrix(y_true=y_test,y_pred=y_pred,normalize='true')

fig = px.imshow(cm, text_auto=True)
fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,40))
tree.plot_tree(dt,feature_names=features_names)

In [ ]:
model_artifact = "model_export"
model_pipeline = Pipeline(steps=[("preprocess",pipeline_preprocessing()),
                            ("train",DecisionTreeClassifier(criterion= 'gini', splitter= 'best',max_depth= 4))])


In [ ]:
X=data.drop(columns=['HeartDisease'])
y=LabelEncoder().fit_transform(data['HeartDisease']).astype(float)
model_pipeline.fit(data,y)


In [ ]:
joblib.dump(model_pipeline, model_artifact) 

In [ ]:
run = wandb.init(project="project_heart",save_code=True)

In [ ]:
# artifact = wandb.Artifact(model_artifact,
#                           type="pipeline_artifact",
#                           description="A full pipeline composed of a Preprocessing Stage and a Decision Tree model"
#                           )

# artifact.add_file(model_artifact)
# run.log_artifact(artifact)

In [ ]:
# artifact = run.use_artifact('diego25rm/project_heart/model_export:v0', type='pipeline_artifact')
# artifact_dir = artifact.download()

In [ ]:
# loaded_model = joblib.load(artifact.file())

In [ ]:
# loaded_model